<a href="https://colab.research.google.com/github/UmaNagirireddi/ML_7/blob/main/Ml_7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, torch, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader; from torchvision import datasets, transforms


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
mean, std = torch.stack([img_t.view(3, -1) for img_t, _ in train_dataset], dim=1).mean(dim=1), torch.stack([img_t.view(3, -1) for img_t, _ in train_dataset], dim=1).std(dim=1)


100%|██████████| 170498071/170498071 [00:03<00:00, 49237348.68it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
mean, std

(tensor([[0.5126, 0.5104, 0.5139,  ..., 0.4972, 0.4963, 0.4966],
         [0.5336, 0.5311, 0.5343,  ..., 0.4929, 0.4926, 0.4935],
         [0.5198, 0.5171, 0.5199,  ..., 0.4464, 0.4467, 0.4486]]),
 tensor([[0.2879, 0.2841, 0.2833,  ..., 0.2508, 0.2517, 0.2546],
         [0.2859, 0.2820, 0.2811,  ..., 0.2415, 0.2425, 0.2457],
         [0.3155, 0.3121, 0.3115,  ..., 0.2545, 0.2557, 0.2591]]))

In [5]:
cifar10 = datasets.CIFAR10('./data', train=True, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))


In [6]:
cifar10_val = datasets.CIFAR10('./data', train=False, download=False,
                               transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))

first_image, label = cifar10[0]
print(first_image.shape)

torch.Size([3, 32, 32])


In [7]:
train_loader, test_loader = [DataLoader(dataset, batch_size=32, shuffle=train, num_workers=2) for dataset, train in zip([cifar10, cifar10_val], [True, False])]
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [33]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1, self.act1, self.pool1 = nn.Conv2d(3, 16, kernel_size=3, padding=1), nn.Tanh(), nn.MaxPool2d(2)
        self.conv2, self.act2, self.pool2 = nn.Conv2d(16, 8, kernel_size=3, padding=1), nn.Tanh(), nn.MaxPool2d(2)
        self.fc1, self.act3, self.fc2 = nn.Linear(8 * 8 * 8, 32), nn.Tanh(), nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool2(self.act2(self.conv2(self.pool1(self.act1(self.conv1(x))))))
        x = x.view(-1, 8 * 8 * 8)
        x = self.fc2(self.act3(self.fc1(x)))
        return x

time: 1.1 ms (started: 2023-12-12 21:38:26 +00:00)


In [35]:
model = Net().to(device)
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18354, [432, 16, 1152, 8, 16384, 32, 320, 10])

time: 15.6 ms (started: 2023-12-12 21:39:43 +00:00)


In [8]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 2 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))



In [45]:
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs = 300, optimizer = optimizer, model = model, loss_fn = loss_fn,train_loader = train_loader,)


2023-12-12 21:58:37.014697 Epoch 1, Training loss 2.0162188311671967
2023-12-12 21:59:18.921000 Epoch 2, Training loss 1.7634930206686639
2023-12-12 22:00:45.500216 Epoch 4, Training loss 1.491888718684311
2023-12-12 22:02:10.591949 Epoch 6, Training loss 1.3718824337815385
2023-12-12 22:03:38.360362 Epoch 8, Training loss 1.2740474026221449
2023-12-12 22:05:03.555960 Epoch 10, Training loss 1.1974983867019644
2023-12-12 22:06:28.593377 Epoch 12, Training loss 1.1374762334177255
2023-12-12 22:07:54.535589 Epoch 14, Training loss 1.0957224607620093
2023-12-12 22:09:19.703955 Epoch 16, Training loss 1.0601110463709478
2023-12-12 22:10:47.887607 Epoch 18, Training loss 1.0291698164952077
2023-12-12 22:12:12.427987 Epoch 20, Training loss 1.0013239630652815
2023-12-12 22:13:37.625106 Epoch 22, Training loss 0.9786256111948691
2023-12-12 22:15:02.892831 Epoch 24, Training loss 0.9567744025336508
2023-12-12 22:16:28.331397 Epoch 26, Training loss 0.9359314717600108
2023-12-12 22:17:54.883259

In [9]:
import collections

In [10]:
trainLoader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)
valLoader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    predictions, exp_labels = [], []

    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct, total = 0, 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(device=device), labels.to(device=device)
                outputs = model(imgs)
                correct += int((torch.max(outputs, 1)[1] == labels).sum())
                total += labels.size(0)

                predictions.extend(outputs.argmax(dim=1).cpu().numpy())
                exp_labels.extend(labels.cpu().numpy())

        print(f"Accuracy {name}: {correct / total:.2f}")
        accdict[name] = correct / total

    return accdict, predictions, exp_labels

In [57]:
accuracy, predictions, expected_labels = validate(model, trainLoader, valLoader)


Accuracy train: 0.79
Accuracy val: 0.61
time: 40.8 s (started: 2023-12-13 01:34:20 +00:00)


In [58]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
precision = precision_score(predictions, expected_labels, average='macro')
recall = recall_score(predictions, expected_labels, average='macro')
cnf_matrix = confusion_matrix(predictions, expected_labels)
print(cnf_matrix)

[[4755  169  259   93   85   43   40   60  360  211]
 [  81 5140   19   26   10   11   14   14  124  383]
 [ 243   32 3912  324  283  217  234  139   81   48]
 [ 124   64  288 3268  197  667  340  141   79   64]
 [ 149   47  550  368 4697  247  260  337   68   60]
 [  63   48  385 1322  226 4440  184  362   33   77]
 [  38   29  274  272  178   88 4845   18   45   27]
 [  96   39  210  234  283  251   38 4867   42  121]
 [ 356  159   88   51   28   15   29   23 5088  175]
 [  95  273   15   42   13   21   16   39   80 4834]]
time: 585 ms (started: 2023-12-13 01:35:29 +00:00)


In [59]:

print(classification_report(predictions, expected_labels, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.79      0.78      0.79      6075
  automobile       0.86      0.88      0.87      5822
        bird       0.65      0.71      0.68      5513
         cat       0.54      0.62      0.58      5232
        deer       0.78      0.69      0.73      6783
         dog       0.74      0.62      0.68      7140
        frog       0.81      0.83      0.82      5814
       horse       0.81      0.79      0.80      6181
        ship       0.85      0.85      0.85      6012
       truck       0.81      0.89      0.85      5428

    accuracy                           0.76     60000
   macro avg       0.76      0.77      0.76     60000
weighted avg       0.77      0.76      0.76     60000

time: 267 ms (started: 2023-12-13 01:35:45 +00:00)


In [11]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4 * 4 * 4, 32)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 4 * 4 * 4)
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out

In [14]:
model2 = Net2().to(device)


In [16]:
import torch.optim as optim

In [17]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model2 = Net2().to(device=device)
optimizer2 = optim.SGD(model2.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer2,
    model = model2,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2023-12-13 02:00:07.617124 Epoch 1, Training loss 2.212731592185662
2023-12-13 02:00:22.718318 Epoch 2, Training loss 1.9845358686678856
2023-12-13 02:00:53.246347 Epoch 4, Training loss 1.722589336240383
2023-12-13 02:01:23.437815 Epoch 6, Training loss 1.5700147395853497
2023-12-13 02:01:55.008355 Epoch 8, Training loss 1.4737615467947158
2023-12-13 02:02:26.318886 Epoch 10, Training loss 1.3993406246995073
2023-12-13 02:02:56.613258 Epoch 12, Training loss 1.3490596872461422
2023-12-13 02:03:26.906569 Epoch 14, Training loss 1.3148994368055593
2023-12-13 02:03:57.147142 Epoch 16, Training loss 1.2820321132459909
2023-12-13 02:04:27.706434 Epoch 18, Training loss 1.253321915026516
2023-12-13 02:04:58.728041 Epoch 20, Training loss 1.2270956954077992
2023-12-13 02:05:28.750207 Epoch 22, Training loss 1.2034590320513987
2023-12-13 02:05:58.742817 Epoch 24, Training loss 1.1855735565390428
2023-12-13 02:06:28.715447 Epoch 26, Training loss 1.1677957881442116
2023-12-13 02:06:58.521277 E

In [18]:
valLoader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)
accuracy2, predictions2, expected_labels2 = validate(model2, trainLoader, valLoader)

Accuracy train: 0.64
Accuracy val: 0.61


In [20]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
precision2 = precision_score(predictions2, expected_labels2, average='macro')
recall2 = recall_score(predictions2, expected_labels2, average='macro')
cnfMatrix2 = confusion_matrix(predictions2, expected_labels2)
print(cnfMatrix2)

[[4373  102  603  179  231   71   76   83 1052  114]
 [ 288 4730   91  106   55   55   62   80  588  426]
 [ 186   21 2271  203  226  157  175   56   48    5]
 [ 103   30  436 2428  358  903  394  193  117   47]
 [ 137   26  699  371 3351  264  304  199   82   28]
 [  56   17  619 1429  276 3551  180  450   42   34]
 [  38   30  572  586  476  194 4572   64   53   29]
 [ 179   48  526  399  884  642   99 4596   54  140]
 [ 204   30   58   69   37   16   36   17 3363   43]
 [ 436  966  125  230  106  147  102  262  601 5134]]


In [21]:
print(classification_report(predictions2, expected_labels2, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.73      0.64      0.68      6884
  automobile       0.79      0.73      0.76      6481
        bird       0.38      0.68      0.49      3348
         cat       0.40      0.48      0.44      5009
        deer       0.56      0.61      0.58      5461
         dog       0.59      0.53      0.56      6654
        frog       0.76      0.69      0.72      6614
       horse       0.77      0.61      0.68      7567
        ship       0.56      0.87      0.68      3873
       truck       0.86      0.63      0.73      8109

    accuracy                           0.64     60000
   macro avg       0.64      0.65      0.63     60000
weighted avg       0.67      0.64      0.65     60000

